In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import sys
import os

# Add the parent directory of the current working directory to sys.path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from params.constants import *
# from loader.loader import *
# import url_builder as ub
import time
import random
from scraping import *

/Users/kevingregory/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
movies = DataLoader("ml_movies").data

In [3]:
DataLoader("ml_ratings").data

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [29]:
def open_page(url: str,
              sleep: bool = True,
              headers: dict = {
                  'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
                  }) -> BeautifulSoup:
    if not url.startswith("https://www.imdb.com/title/tt"):
        raise ValueError("Invalid URL")
    
    response = requests.request("GET", url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content)
    
    if sleep:
        time.sleep(random.randint(1, 5))
    
    return soup

In [36]:
def get_review_url(soup: BeautifulSoup) -> (str, str):
    section = soup.find('section', {'data-testid': 'UserReviews'})
    if section:
        a_tag = section.find('a')
        
        if a_tag:
            return "https://www.imdb.com" + a_tag['href']
        else:
            return "Hyperlink not found within the specified section."
    else:
        return "Section with the specified data-testid not found."

In [53]:
review_soup = BeautifulSoup(reviews_page, "html.parser")

TypeError: 'NoneType' object is not callable

In [2]:
all_reviews = {}
for ids in ["0114709","0499549"]:
    all_reviews = all_reviews | get_reviews(ids)

/Users/kevingregory/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.imdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py:21: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 21 of the file /Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(

In [3]:
all_reviews

{'0114709': ["Toy Story is not only the best Disney film because it has the best story and the best animation, but also because of the excellent actors chosen to provide the voices of the animals. The casting was perfect from top to bottom, and the movie provides an excellent adventure story about friendship and loyalty that keeps you engrossed until the nail-biting climax.\nTom Hanks and Tim Allen provided excellent voices for Woody and Buzz Lightyear -their performances alone are one of the biggest things that made this such a spectacular movie. Besides that, though, you have the excellent story that is not only noteworthy because it has never really been told from this perspective before, but also because it was just told so well. All of the characters in the film are very well developed and all have appropriate and effective actors chosen to provide their voices. And of course, who could forget the revolutionary animation! The computer animation used for this movie not only made it

In [3]:
reviews = get_reviews('113497')

/Users/kevingregory/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.imdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py:21: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 21 of the file /Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(

In [4]:
type(reviews)

list

In [3]:
# url = imdbId_to_url(test_row_links["imdbId"])
url = "https://www.imdb.com/title/tt0113497/"
print(url)

https://www.imdb.com/title/tt0113497/


In [7]:
test_row_links["imdbId"]

'113497'

In [7]:
get_review_url('113497')

'https://www.imdb.com/title/tt0113497/reviews?sort=totalVotes&dir=desc&ratingFilter=0'

In [5]:
review_soup = open_page(get_review_url('113497'))

/Users/kevingregory/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.imdb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py:21: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 21 of the file /Users/kevingregory/Desktop/programs/Movie_Recommendation/scraping/scraping.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(

In [9]:
text_divs = review_soup.find_all('div', class_='text show-more__control')

In [10]:
type(text_divs)

bs4.element.ResultSet

In [13]:
text_divs[0].text

"Among the thousands of films I have viewed, this movie would rank near the top for sheer entertainment. That's not saying it's the best-made or most intelligent or scariest or funniest or features the best effects, etc. etc. But combine all those and you have a film that's tough to beat when you're looking for 100 minutes of escapist fun.The film features some wild computer-enhanced special effects that were new to its day, but now about 10 years later, it's no big deal. In fact, some of it, such as the lion, look pretty hokey compared to the stuff that's out there now. To me, it was story that was the lure, anyway, not the special effects.Because it's so much fun, this is one of the fastest-moving films I've ever viewed. The time flies by. It's not to be analyzed or given much thought, because it's so ludicrous. You just go along for the wild ride in this fantasy-adventure and get a bunch of laughs and thrills along the way. That's one of the big attractions of this of film: the exce